In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
from difflib import SequenceMatcher
import itertools

import cmudict
from gensim.models import Word2Vec

import numpy as np
import pandas as pd

### Word2Vec model to identify similar sounds

In [3]:
flatten = lambda l: [item for sublist in l for item in sublist]

In [4]:
# NOTE: Do we want to remove emphasis?
words = flatten(list(cmudict.dict().values()))
phones = np.unique(flatten(words))

In [5]:
model = Word2Vec(words, size=10, window=1, min_count=1, workers=8)

### Functions for computing Levenshtein distance using W2V similarity

In [6]:
# TODO: default to double metaphone if either word NOT in CMU dict
def add_pronunciation(function):
    def wrapper(w1, w2, w2v):
        w1_pron = cmudict.dict()[w1]
        w2_pron = cmudict.dict()[w2]
        pairs = itertools.product(w1_pron, w2_pron)
        return max([function(i, j, w2v) for i, j in pairs])
    return wrapper

In [7]:
# levenshtein edit distance function
def edit_distance_(w1, w2, w2v):
    
    cost = []
    
    for i in range(len(w1)+1):
        x = []
        for j in range(len(w2)+1):
            x.append(0)
        cost.append(x)
    
    for i in range(len(w1)+1):
        cost[i][0] = i
    for j in range(len(w2)+1):
        cost[0][j] = j
        
    # baseline costs
    del_cost = 1
    add_cost = 1
    
    for i in range(1, len(w1)+1):
        for j in range(1, len(w2)+1):
            if w1[i-1] == w2[j-1]:
                sub_cost = 0
            else:
                sub_cost = 1 - w2v.wv.similarity(w1[i-1], w2[j-1])
            # get the totals
            del_total = cost[i-1][j] + del_cost
            add_total = cost[i][j-1] + add_cost
            sub_total = cost[i-1][j-1] + sub_cost
            # choose the lowest cost from the options
            options = [del_total, add_total, sub_total]
            options.sort()
            cost[i][j] = options[0]

    ratio = (1 - cost[-1][-1]/max(len(w1), len(w2)))
            
    return ratio

In [8]:
# return ratio of most similar substring
def partial_edit_distance_(w1, w2, w2v):

    if len(w1) <= len(w2):
        shorter = w1
        longer = w2
    else:
        shorter = w2
        longer = w1

    m = SequenceMatcher(None, shorter, longer)
    blocks = m.get_matching_blocks()

    scores = []
    for block in blocks:
        long_start = block[1] - block[0] if (block[1] - block[0]) > 0 else 0
        long_end = long_start + len(shorter)
        long_substr = longer[long_start:long_end]

        r = edit_distance_(shorter, long_substr, w2v)
        scores.append(r)

    return max(scores)

In [9]:
@add_pronunciation
def edit_distance(*args, **kwargs):
    return edit_distance_(*args, **kwargs)

@add_pronunciation
def partial_edit_distance(*args, **kwargs):
    return partial_edit_distance_(*args, **kwargs)

In [10]:
partial_edit_distance('koala', 'qualification', model)

0.746347326040268

In [11]:
partial_edit_distance('relevant', 'elephant', model)

0.911413597209113

In [12]:
partial_edit_distance('humanity', 'manatee', model)

0.9193509519100189

In [13]:
partial_edit_distance('iguana', 'wanna', model)

1.0

In [14]:
partial_edit_distance('ribbit', 'rivet', model)

0.904794430732727